# LIBRERIAS

In [3]:
import os
import scipy
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.io import loadmat
from scipy.interpolate import splev, splrep
import scipy.io as sio
from scipy.signal import resample
import seaborn as sns
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, f1_score

# CARGA DEL ARCHIVO CON LOS SEGMENTOS FILTRADOS

In [ ]:
loaded_data = np.load('filtrado_70mv.npz', allow_pickle=True)

In [ ]:
#Hacemos comprobaciones

total_length = 0

# Iterar sobre cada archivo en el .npz
for key in loaded_data.files:
    signal_dict = loaded_data[key].item()
    dict_length = len(signal_dict)
    
    
    # Sumar la longitud del diccionario actual al total acumulado
    total_length += dict_length

# Imprimir la suma total de las longitudes de los diccionarios
print("Total length:", total_length)


# MÉTODOS

## MÉTODO ESPECTRAL

### FUNCIONES

A) FUNCIÓN MÉTODO ESPECTRAL

In [ ]:
def SM_TWA(M_Wind):
"""
Esta función aplica el método espectral mejorado (SM) explicado en:

MG Fernández-Calvillo et al., "Machine Learning approach for TWA detection relying on ensemble data design," Heliyon, Vol. 9, no. 1, enero 2023.

El método puede trabajar tanto con complejos ST-T como con complejos ST-T diferenciales. El SM estima la densidad espectral de potencia de cada serie de latidos
Sj mediante el periodograma:

             Pj=(1/M)|FFT({Sj})|^2

donde M representa el número de latidos tomados para aplicar el SM. El espectro de potencia promedio de todas las series de latidos es:

                 P=(1/N)∑Pj

Parámetros:
    M_Wind (array): Array que contiene la señal correspondiente a una ventana.

Retornos:
    AverPSD (array): Espectro promedio.
    K_score (float): Valor de TWAR.
    Valt (float): Onda alternante estimada.
"""

    M = len(M_Wind) 
    N = len(M_Wind[0]) 
    L = int(2 ** np.ceil(np.log2(M)))  # Resolución de la FFT.
  


    PSD_BeatSeries = np.abs(np.fft.fft(M_Wind, L, axis=0)) ** 2
    AverPSD = np.mean(PSD_BeatSeries, axis=1) / (M * N)

    noise_band = AverPSD[int(0.66 * ((L / 2)-1)):int(0.96 * ((L / 2)-1))]


    noise_mean = np.mean(noise_band)

    noise_std = np.std(noise_band)

    T = AverPSD[1 + int(((L / 2)-1))]

    K_score = (T - noise_mean) / noise_std

    if K_score > 0:
        Valt = np.sqrt(T - noise_mean) / np.sqrt(M)   # La división por sqrt(M) sirve para compensar un factor de ponderación.
    else:
        Valt = 0



    return AverPSD, K_score, Valt


B) FUNCIÓN DE CLASIFICIÓN

In [ ]:
def assign_labels(K_score):
    labels = [] # Lista para almacenar los 0 o 1

    for i in K_score:
        if i < 3:  # el umbral se establece en 3 
            labels.append(0) # no posee alternancia se agrega a la lista como 0 porque el kscore es menor que 3
        else:
            labels.append(1) # posee alternancia, se agrega a la lista como 1 porque el kscore es mayor que 3

    return labels

C) FUNCION DE EVALUACIÓN

In [ ]:
def evaluate_performance(true_labels, predicted_labels):
    confusion_mat = confusion_matrix(true_labels, predicted_labels) # MAtriz de confusión
    precision = precision_score(true_labels, predicted_labels) # Cálculo precisión
    accuracy = accuracy_score(true_labels, predicted_labels) # Cálculo exactitud
    recall = recall_score(true_labels, predicted_labels) # Cálculo sensibilidad
    f1 = f1_score(true_labels, predicted_labels) # Cálculo f1 score

    return confusion_mat, precision, accuracy, recall, f1

### APLICACIÓN DEL MÉTODO EN LA BASE DE DATOS

In [ ]:
K_scores = []
for key in loaded_data.files:
    signal_dict = loaded_data[key].item()
    for sub_key, signal_array in signal_dict.items():
        AverPSD, k_score, Valt = SM_TWA(signal_array) # Aplicamos método espectral
        K_scores.append(k_score) # se obtienen los k-score




### EVALUACIÓN

In [ ]:
labels = assign_labels(K_scores)
# contar cuantos segmentos tienen alternancia y cuántos no
count_0 = labels.count(0)
count_1 = labels.count(1)

print("Number of 0 labels:", count_0) 
print("Number of 1 labels:", count_1)

In [ ]:
# Creamos las etiquetas verdaderas de la base de datos, sabiendo que la mitad son con alternancia y la otra mitad si alternancia

true_labels = np.concatenate([np.ones((total_length) // 2), np.zeros((total_length) // 2)])


In [ ]:
#Obtenemos las métricas

confusion_mat, precision, accuracy, recall, f1 = evaluate_performance(true_labels, labels)

print("Confusion Matrix:")
print(confusion_mat)
print("Precision:", precision)
print("Accuracy:", accuracy)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
# Crear el gráfico
plt.figure(figsize=(7, 5))
sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Blues', xticklabels=['No Alternancia', 'Alternancia '], yticklabels=['No Alternancia', 'Alternancia '])
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Verdadera')
plt.title('Matriz de Confusión del Método Espectral (70 µV)')
plt.show()

## MÉTODO TEMPORAL

### FUNCIONES

In [ ]:
import numpy as np

def TimeMethod_TWA(M_Wind):
"""
Esta función aplica el Método del Tiempo para detectar TWA, tal como se informa en:

MG Fernández-Calvillo et al., "Machine Learning approach for TW detection relying on ensemble data design," Heliyon, Vol. 9, No. 1, enero 2023.

Parámetros:
    M_Wind (array): Array que contiene la señal correspondiente a una ventana de complejos ST-ST.

Retornos:
    Valt_est (float): Voltaje TWA estimado (en microV).
    Valt_wave_est (array): Estimación de ML de la onda alternante (en microV).
"""
    M_Wind = np.array(M_Wind)  # Convierte la lista en NumPy array 

    Valt_wave_est = np.mean(M_Wind[::2, :], axis=0) - np.mean(M_Wind[1::2, :], axis=0)
    Valt_est = np.max(np.abs(Valt_wave_est)) * 0.5 * 1e3

    return Valt_est, Valt_wave_est

### APLICACIÓN DEL MÉTODO EN LA BASE DE DATOS

In [ ]:
Valt_TM_dict=[]
for key in loaded_data.files:
    signal_dict = loaded_data[key].item()
    for sub_key, signal_array in signal_dict.items():

        Valt_TM_est, _ = TimeMethod_TWA(signal_array)
        Valt_TM_dict.append(Valt_TM_est)


### EVALUACIÓN

In [ ]:

# Cáculo de la media y la desviación estándar para el subconjunto con alternancia
mean_1st_half = np.mean(Valt_TM_dict[:len(Valt_TM_dict)//2 ])
std_1st_half = np.std(Valt_TM_dict[:len(Valt_TM_dict)//2 ])

# Cáculo de la media y la desviación estándar para el subconjunto sin alternancia
mean_2nd_half = np.mean(Valt_TM_dict[len(Valt_TM_dict)//2:])
std_2nd_half = np.std(Valt_TM_dict[len(Valt_TM_dict)//2 :])

print("Primera Mitad:")
print("Amplitud Media:", mean_1st_half)
print("Deviación Estándar:", std_1st_half)

print("Segunda Mitad:")
print("Amplitud Media:", mean_2nd_half)
print("Deviación Estándar:", std_2nd_half)

# Comparamos la media y la desviación
if mean_1st_half > mean_2nd_half:
    print("La amplitud media es mayor en la primera mitad:", mean_1st_half )
else:
    print("La amplitud media es mayor en la segunda mitad", mean_2nd_half)

if std_1st_half > std_2nd_half:
    print("La desviación estándar es mayor en la primera mitad:", std_1st_half)
else:
    print("La desviación estándar es mayor en la segunda mitad:", std_2nd_half)

In [ ]:
group_with_alternans = Valt_TM_dict[:len(Valt_TM_dict)//2]
group_without_alternans = Valt_TM_dict[len(Valt_TM_dict)//2:]

# Creamos una lista con los dos subconjuntos
data = [group_with_alternans, group_without_alternans]

# Crear la figura y los ejes
fig, ax = plt.subplots()

# Creamos el boxplot
boxplot = ax.boxplot(data, patch_artist=True, notch=True)

# Establecer los colores del boxplot
colors = ['lightblue', 'lightgreen']
for patch, color in zip(boxplot['boxes'], colors):
    patch.set_facecolor(color)

# Agregar etiquetas y título
ax.set_xticklabels(['Grupo con Alternancias', 'Grupo sin Alternancias'])
ax.set_ylabel('Amplitud (µV)')
plt.title('Distribución de la Amplitud a 70µV')
plt.axhline(70) #umbral dependiendo del voltaje de amplitud

plt.show()

## MÉTODO  DE LA RAZÓN DE PROBABILIDAD LAPLACIANA

### FUNCIONES

A) FUNCIÓN MÉTODO DE LA RAZÓN DE PROBABILIDAD LAPLACIANA

In [ ]:
def LLR_method(M_Wind, D):
# Esta función aplica el método de la Razón de Verosimilitud Laplaciana para detectar TWA.
# Reportado en: Juan Pablo Martínez et al., "Characterization of repolarization alternans during ischemia: Time-Course and spatial analysis," IEEE Trans Bio Eng, Vol. 53, No. 4, abril 2006.
#
# Parámetros de entrada:
#   - M_Wind: array que contiene la señal correspondiente a una ventana.
#   - D: tasa de reducción de datos, se propone establecerla en 8.
# Parámetros de salida:
#   - Valt_est: voltaje TWA estimado (en microV).
#   - Valt_wave_est: estimación de ML de la onda alternante (en microV).
#   - T: parámetro de significancia estadística.

    X_decimated = resample(M_Wind.T, int(len(M_Wind.T)/D)).T # Aplicamos un diezmado 
    M = M_Wind.shape[0]  # Dimensión de latido 
    P = X_decimated.shape[1]
    Alt_matrix = np.tile(((-np.ones(M))**(np.arange(M))),(P,1)).T
    CompDem_X = X_decimated* Alt_matrix
    Valt_est_wave = np.median(CompDem_X, axis=0)
    Valt_est = np.sqrt(np.mean(Valt_est_wave**2))* 0.5 * 1e3
    T_series = np.zeros(Valt_est_wave.size)

    for p in range(len(T_series)):
        aux = CompDem_X[:, p]
        if Valt_est_wave[p] < 0:
            T_series[p] = 2 * np.sum(np.abs(aux[(aux < 0) & (aux > Valt_est_wave[p])]))
        else:
            T_series[p] = 2 * np.sum(abs(aux[(aux > 0) & (aux < Valt_est_wave[p])]))
                  
    MLE_noise = np.sum(np.sum(np.abs(CompDem_X - Valt_est_wave)))
    T = np.sum(T_series) / MLE_noise

    
    return Valt_est, Valt_est_wave, T

B) FUNCIÓN DE CLASIFICIÓN

In [ ]:
def assign_labels_LLR(T):
    labels_LLR = [] # Lista para almacenar los 0 o 1

    for T in T_series:
        if T < 0.15: # el umbral se establece en 0.15
            labels_LLR.append(0)# no posee alternancia, se agrega a la lista como 0 porque el T es menor que 0.15
        else:
            labels_LLR.append(1) # posee alternancia, se agrega a la lista como 1 porque T es mayor que 0.15

    return labels_LLR

### APLICACIÓN DEL MÉTODO EN LA BASE DE DATOS

In [ ]:
Valt_est_dict=[]
T_series=[]
for key in loaded_data.files:
    signal_dict = loaded_data[key].item()
    
    for sub_key, signal_array in signal_dict.items():
        Valt_est,  Valt_est_wave, T = LLR_method(signal_array,1)
        T_series.append(T)
        Valt_est_dict.append(Valt_est)

### EVALUACIÓN

a) Estimación

In [ ]:
group_with_alternans = Valt_est_dict[:len(Valt_est_dict)//2]
group_without_alternans = Valt_est_dict[len(Valt_est_dict)//2:]

# Creamos una lista con los dos subconjuntos
data = [group_with_alternans, group_without_alternans]

# Crear la figura y los ejes
fig, ax = plt.subplots()

# Create the boxplot
boxplot = ax.boxplot(data, patch_artist=True, notch=True)

# Creamos el boxplot
colors = ['lightblue', 'lightgreen']
for patch, color in zip(boxplot['boxes'], colors):
    patch.set_facecolor(color)

# Establecer los colores del boxplot
ax.set_xticklabels(['Grupo con Alternancias', 'Grupo sin Alternancias'])
ax.set_ylabel('Amplitud (µV) ')
plt.title('Distribución de la Amplitud a 70µV')
plt.axhline(70)  #umbral dependiendo del voltaje de amplitud

plt.show()

In [ ]:
# Cáculo de la media y la desviación estándar para el subconjunto con alternancia
mean_1st_half = np.mean(Valt_est_dict[:len(Valt_est_dict)//2 ])
std_1st_half = np.std(Valt_est_dict[:len(Valt_est_dict)//2 ])

# Cáculo de la media y la desviación estándar para el subconjunto sin alternancia
mean_2nd_half = np.mean(Valt_est_dict[len(Valt_est_dict)//2:])
std_2nd_half = np.std(Valt_est_dict[len(Valt_est_dict)//2 :])

print("Primera Mitad:")
print("Amplitud Media:", mean_1st_half)
print("Deviación Estándar:", std_1st_half)

print("Segunda Mitad:")
print("Amplitud Media:", mean_2nd_half)
print("Deviación Estándar:", std_2nd_half)

# Comparamos la media y la desviación
if mean_1st_half > mean_2nd_half:
    print("La amplitud media es mayor en la primera mitad:", mean_1st_half )
else:
    print("La amplitud media es mayor en la segunda mitad", mean_2nd_half)

if std_1st_half > std_2nd_half:
    print("La desviación estándar es mayor en la primera mitad:", std_1st_half)
else:
    print("La desviación estándar es mayor en la segunda mitad:", std_2nd_half)

b) Detección

In [ ]:
labels_LLR = assign_labels_LLR(T_series)
# contar cuantos segmentos tienen alternancia y cuántos no
count_0_LLR = labels_LLR.count(0)
count_1_LLR = labels_LLR.count(1)

print("Number of 0 labels:", count_0_LLR)
print("Number of 1 labels:", count_1_LLR)

In [ ]:
#Obtenemos métricas

confusion_mat, precision, accuracy, recall, f1 = evaluate_performance(true_labels, labels_LLR)

print("Confusion Matrix:")
print(confusion_mat)
print("Precision:", precision)
print("Accuracy:", accuracy)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
# Crear el gráfico
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Reds', xticklabels=['No Alternancia', 'Alternancia '], yticklabels=['No Alternancia', 'Alternancia '])
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Verdadera')
plt.title('Matriz de Confusión del Método de la Razón de Probabilidad Laplaciana(70 µV)')
plt.show()

## MÉTODO DE PROMEDIO MÓVIL MODIFICADO

### FUNCIONES

In [ ]:
def MMA_TWA(x, First_2_ST_estimates, WindStep):
    # Inicializamos las variables
    M, N = x.shape
    x_est = np.zeros((M + 2, N))
    x_est[0:2, :] = First_2_ST_estimates[0:2, :]
    Fraction_error_estimate = 8

    # Estimamos el segmento ST
    for k1 in range(2, M + 2):
        e = (x[k1 - 2, :] - x_est[k1 - 2, :]) / Fraction_error_estimate
        h = np.zeros(N)
        for k2 in range(N):
            if e[k2] <= -0.032:
                h[k2] = -0.032
            elif -0.032 < e[k2] <= -0.001:
                h[k2] = e[k2]
            elif -0.001 < e[k2] < 0:
                h[k2] = -0.001
            elif e[k2] == 0:
                h[k2] = 0
            elif 0 < e[k2] <= 0.001:
                h[k2] = 0.001
            elif 0.001 < e[k2] <= 0.032:
                h[k2] = e[k2]
            else:
                h[k2] = 0.032
        x_est[k1, :] = x_est[k1 - 2, :] + h

    # Calculamos las salidas
    Next_2_ST_estimates = x_est[WindStep:WindStep + 2, :]
    x_est = x_est[:M, :]

    aux = np.diff(x_est, axis=0)  # Determinar la primera diferencia por filas
    V_alt_MMA = aux[::2, :] * 1e3  # En microvoltios
    V_MMA = np.max(np.abs(V_alt_MMA), axis=1)

    return V_MMA, V_alt_MMA, Next_2_ST_estimates

### APLICACIÓN DEL MÉTODO EN LA BASE DE DATOS

In [ ]:
WindStep= 8
MethPred_Predictor_dict=[]

for key in loaded_data.files:
    signal_dict = loaded_data[key].item()
    for sub_key, signal_array in signal_dict.items():
        x_est_input = np.zeros((2, signal_array.shape[1]))
        x = signal_array
        V, _, x_est_out = MMA_TWA(x, x_est_input, WindStep)
        MethPred_Predictor_dict.append(V[-1])
        x_est_input = x_est_out

### EVALUACIÓN

In [ ]:
# Cáculo de la media y la desviación estándar para el subconjunto con alternancia
mean_1st_half = np.mean(MethPred_Predictor_dict[:len(MethPred_Predictor_dict)//2 ])
std_1st_half = np.std(MethPred_Predictor_dict[:len(MethPred_Predictor_dict)//2 ])

# Cáculo de la media y la desviación estándar para el subconjunto sin alternancia
mean_2nd_half = np.mean(MethPred_Predictor_dict[len(MethPred_Predictor_dict)//2:])
std_2nd_half = np.std(MethPred_Predictor_dict[len(MethPred_Predictor_dict)//2 :])

print("Primera Mitad:")
print("Amplitud Media:", mean_1st_half)
print("Deviación Estándar:", std_1st_half)

print("Segunda Mitad:")
print("Amplitud Media:", mean_2nd_half)
print("Deviación Estándar:", std_2nd_half)

# Comparamos la media y la desviación
if mean_1st_half > mean_2nd_half:
    print("La amplitud media es mayor en la primera mitad:", mean_1st_half )
else:
    print("La amplitud media es mayor en la segunda mitad", mean_2nd_half)

if std_1st_half > std_2nd_half:
    print("La desviación estándar es mayor en la primera mitad:", std_1st_half)
else:
    print("La desviación estándar es mayor en la segunda mitad:", std_2nd_half)

In [ ]:
group_with_alternans = MethPred_Predictor_dict[:len(MethPred_Predictor_dict)//2]
group_without_alternans = MethPred_Predictor_dict[len(MethPred_Predictor_dict)//2:]

# Creamos una lista con los dos subconjuntos
data = [group_with_alternans, group_without_alternans]

# Crear la figura y los ejes
fig, ax = plt.subplots()

# Crear un boxplot
boxplot = ax.boxplot(data, patch_artist=True, notch=True)

# Establecer los colores del boxplot
colors = ['lightblue', 'lightgreen']
for patch, color in zip(boxplot['boxes'], colors):
    patch.set_facecolor(color)

# Añadimos el título y los ejes de coordenadas
ax.set_xticklabels(['Grupo con Alternancias', 'Grupo sin Alternancias'])
ax.set_ylabel('Amplitud µV')
plt.title('Distribución de la Amplitud a 70µV')
plt.axhline(70)#establecemos el umbral dependiendo del voltaje de amplitud
plt.show()

## MÉTODO DE CORRELACIÓN

A) FUNCIÓN MÉTODO DE CORRELACIÓN

In [ ]:
def Correlation_Method_TWA(M_matrix):
    Tmdn = np.median(M_matrix, axis=0)  # Mediana de la onda T: en toda la ventana
    ACI = np.sum(M_matrix * Tmdn, axis=1) / np.sum(Tmdn ** 2)  # Índice de correlación de alternancia (ACI)
    ACI_1_wind = ACI - 1
    Acm_wind = 2 * (np.abs(ACI_1_wind)) * ((np.sum(Tmdn ** 2)) / (np.sum(np.abs(Tmdn))))  # Estimación de la amplitud de TWA

    # Función para calcular la tasa de cruces por cero
    def zerocrossrate(signal):
        crossings = [0] * len(signal)
        for i in range(1, len(signal)):
            if (signal[i - 1] >= 0 and signal[i] < 0) or (signal[i - 1] < 0 and signal[i] >= 0):
                crossings[i] = 1
        return crossings

    indices = zerocrossrate(ACI_1_wind)
    indices_restados = np.diff(np.array([0] + list(indices) + [0]))
    f = np.where((indices_restados == 1) | (indices_restados == -1))[0]
    consec_count = f[1::2] - f[::2]

    if consec_count.size > 0:
        zcr_max_wind = np.max(consec_count)
    else:
        zcr_max_wind = 0  # O cualquier otro valor predeterminado que tenga sentido en tu contexto

    return zcr_max_wind, Acm_wind

B) FUNCIÓN DE CLASIFICACIÓN

In [ ]:
def assign_labels_CM(zcr_max):
    labels_CM = [] # Lista para almacenar los 0 o 1

    for zcr_max in zcr_max_dict:
        if zcr_max < 5:  # el umbral se establece en 5
            labels_CM.append(0)# no posee alternancia, se agrega a la lista como 0 porque el zc_max es menor que 5
        else:
            labels_CM.append(1)# posee alternancia, se agrega a la lista como 1 porque el zc_max es mayor que 5

    return labels_CM

### APLICACIÓN DEL MÉTODO EN LA BASE DE DATOS


In [ ]:
zcr_max_dict = [] 

for key in loaded_data.files:
    signal_dict = loaded_data[key].item()
    for sub_key, signal_array in signal_dict.items():
        zcr_max,_ = Correlation_Method_TWA(signal_array)
        zcr_max_dict.append(zcr_max)



### EVALUACIÓN

In [ ]:
labels_CM = assign_labels_CM(zcr_max)
# contar cuantos segmentos tienen alternancia y cuántos no
count_0_CM = labels_CM.count(0)
count_1_CM = labels_CM.count(1)

print("Number of 0 labels:", count_0_CM)
print("Number of 1 labels:", count_1_CM)


In [ ]:
#Obtenemos métricas

confusion_mat, precision, accuracy, recall, f1 = evaluate_performance(true_labels, labels_CM)

print("Confusion Matrix:")
print(confusion_mat)
print("Precision:", precision)
print("Accuracy:", accuracy)
print("Recall:", recall)
print("F1-score:", f1)

In [ ]:
# Crear el gráfico
plt.figure(figsize=(8, 6))
sns.heatmap(confusion_mat, annot=True, fmt='d', cmap='Greens',xticklabels=['No Alternancia', 'Alternancia '], yticklabels=['No Alternancia', 'Alternancia '])
plt.xlabel('Predicción')
plt.ylabel('Etiqueta Verdadera')
plt.title('Matriz de Confusión del Método de Correlación (70 µV)')
plt.show()